# Fitting the orbit of an exoplanet host star using simulated Gaia astrometry time series

### On the basis of https://zenodo.org/records/7081002 with contributions by B. Holl and J.-B. Delisle

### Imports

In [ ]:
from collections import OrderedDict
import copy
import glob
import os

import astropy.units as u
from astroquery.gaia import Gaia
from astropy.time import Time
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd

from kepmodel.astro import AstroModel as AstrometricModel
import spleaf
from pystrometry.pystrometry import convert_from_angular_to_linear, pjGet_m2, OrbitSystem, MS_kg, MJ_kg
from pystrometry import gaia_astrometry

### A helper function for plotting

In [ ]:
def plot_astrometry(simulated_astrometry, parameter_df, plot_dir, reference_time, model_name):
    """Make a figure showing the model and the data."""
    
    source_id = parameter_df.loc[0, 'source_id']

    iad = gaia_astrometry.GaiaValIad.from_dataframe(simulated_astrometry, source_id, 'source_id')

    # iad.filter_on_strip(strip_threshold=2)
    iad._mjd_field = 'mjd'
    iad.time_column = 'relative_time_year'
    iad.set_reference_time(reference_time)

    parameter_dict = parameter_df[parameter_df['source_id'] == source_id].iloc[0].to_dict()
    parameter_dict['source_id'] = np.int64(source_id)
    parameter_dict['name_seed'] = f"{source_id}_{model_name}"
    if model_name == 'single_star':
        axp = gaia_astrometry.plot_individual_ppm(parameter_dict, iad, plot_dir)
    elif model_name == 'multiple_star':
        axp = gaia_astrometry.plot_individual_orbit(parameter_dict, iad, mapping_dr3id_to_starname=None, plot_dir=plot_dir, m1_MS=parameter_dict['m1_MS'])

In [ ]:
base_dir = os.getcwd()
plot_dir = os.path.join(base_dir)

### Load simulated astrometry data

In [ ]:
data_file = 'simulated_astrometry.csv'
simulated_astrometry = pd.read_csv(data_file)
display(simulated_astrometry[['Target', 'source_id', 'relative_time_year', 'scanAngle[rad]', 'centroidPosAl_mas', 'centroidPosAlError_mas']])

## Fit the Keplerian astrometric model using [kepmodel](https://obswww.unige.ch/~delisle/kepmodel/doc/) (https://ui.adsabs.harvard.edu/abs/2022A%26A...667A.172D/abstract)
The astrometric motion corresponding to a Keplerian orbit of a binary system has generally seven independent parameters. These
are the period $P$, the epoch of periastron passage $T_0$, the eccentricity $e$, the inclination $i$, the ascending node $\Omega$, the argument of periastron $\omega$, and the semi-major axis of the photocentre $a_0$.
The Thiele-Innes coefficients $A, B, F, G$, which linearise part of the equations are defined as described e.g. in https://ui.adsabs.harvard.edu/abs/2023A%26A...674A..10H/abstract 

<!-- \begin{eqnarray}
\begin{split}
 \begin{array}{ll}
A &=&  \ \ \, a_0 \; (\cos \omega \cos \Omega - \sin \omega \sin \Omega \cos i)   \\
B &=&  \ \ \, a_0 \; (\cos \omega \sin \Omega + \sin \omega \cos \Omega \cos i)  \\
F &=& -a_0 \; (\sin \omega \cos \Omega + \cos \omega \sin \Omega \cos i)  \\
G &=&  -a_0 \; (\sin \omega \sin \Omega - \cos \omega \cos \Omega \cos i) 
 \end{array}
\end{split}
\label{eq:cu4nss_astrobin_orbital_ABFG}
\end{eqnarray}
The elliptical rectangular coordinates $X$ and $Y$ are functions of eccentric anomaly $E$ and eccentricity: 
\begin{eqnarray}
E - e \sin E &=& \frac{2\pi}{P} (t-T_0)\\
X &=& \cos E - e\\
Y &=& \sqrt{1-e^2} \sin E
\end{eqnarray}
The single Keplerian model can then be written as 
\begin{equation}\label{eq:k1_model}
w_\mathrm{k1} = (B \, X + G \, Y) \sin \psi + (A \, X + F \, Y) \cos \psi.
\end{equation}
The combined model $w^\mathrm{(model)}$ for the Gaia along-scan abscissa is

\begin{equation}\label{eq:abscissa1}
\begin{split}
w^\mathrm{(model)} =&\, w_\mathrm{ss} + w_\mathrm{k1} \\
 =&\, ( \Delta\alpha^{\star} + \mu_{\alpha^\star} \, t ) \, \sin \psi + ( \Delta\delta + \mu_\delta \, t ) \, \cos \psi + \varpi \, f_\varpi \\
 &+\, (B \, X + G \, Y) \sin \psi + (A \, X + F \, Y) \cos \psi.
\end{split}\end{equation}

Standard model:
\begin{equation}\label{eq:single_source_model}
w_\mathrm{ss} = ( \Delta\alpha^{\star} + \mu_{\alpha^\star} \, t ) \, \sin \psi + ( \Delta\delta + \mu_\delta \, t ) \, \cos \psi + \varpi \, f_\varpi, 
\end{equation}
 -->



### First fit the single-star model

In [ ]:
simulated_astrometry['relative_time_day'] = simulated_astrometry['relative_time_year'] * u.year.to(u.day)

single_star_model = AstrometricModel(simulated_astrometry['relative_time_day'].values, 
                                     simulated_astrometry['centroidPosAl_mas'].values, 
                                     simulated_astrometry['cpsi_obs'].values, 
                                     simulated_astrometry['spsi_obs'].values, 
                                     err=spleaf.term.Error(simulated_astrometry['centroidPosAlError_mas'].values),
                                     jit=spleaf.term.Jitter(0.05))

print("=== Single star model ===")
single_star_model.add_lin(simulated_astrometry['spsi_obs'].values, 'ra')
single_star_model.add_lin(simulated_astrometry['cpsi_obs'].values, 'dec')
single_star_model.add_lin(simulated_astrometry['parallaxFactorAlongScan'].values, 'parallax')
single_star_model.add_lin(simulated_astrometry['relative_time_year'].values * simulated_astrometry['spsi_obs'].values, 'mura')
single_star_model.add_lin(simulated_astrometry['relative_time_year'].values * simulated_astrometry['cpsi_obs'].values, 'mudec')
single_star_model.fit_param += ['cov.jit.sig'] # add a jitter term
single_star_model.fit()
single_star_model.show_param()

## Add the Keplerian motion to the model

In [ ]:
model = copy.deepcopy(single_star_model)

# Periodogram settings
Pmin = 5
Pmax = 50000
nfreq = 10000
nu0 = 2 * np.pi / Pmax
dnu = (2 * np.pi / Pmin - nu0) / (nfreq - 1)
fap_threshold = 1e-3

for kpla in range(1):
    res = model.residuals()
    res_err = np.sqrt(model.cov.A)
    pl.figure()
    pl.errorbar(model.t,res, yerr=res_err, fmt='.', rasterized=True, ecolor='lightgrey')
    pl.ylabel('Residuals (mas)')
    pl.show()    
    nu, power = model.periodogram(nu0, dnu, nfreq)
    P = 2 * np.pi / nu
    
    kmax = np.argmax(power)
    faplvl = model.fap(power[kmax], nu.max())

    pl.figure(figsize=(10, 3))
    pl.plot(2 * [P[kmax]], [0, 1.1 * power.max()], 'r')
    pl.plot(P, power, 'k', lw=1.2, rasterized=True)
    pl.xlim(Pmin, Pmax)
    pl.ylim(0, 1.1 * power.max())
    pl.xscale('log')
    pl.xlabel('Period (d)')
    pl.ylabel('Normalized power')
    pl.title(f'P={P[kmax]:.3f} d' + 20*' ' + f'FAP={faplvl:.2g}')
    # pl.axhline(fap_threshold, ls='--', label='fap_threshold')
    pl.show()
    
    if faplvl > fap_threshold:
        print('No more significant peak')
        break
    print(f'Add a planet at {P[kmax]:.3f} d:')
    model.add_keplerian_from_period(P[kmax])
    model.fit()
    model.show_param()
    
    print('Change orbital parameter set:')
    param = ['P', 'Tp', 'as', 'e', 'w', 'i', 'bigw']
    model.set_keplerian_param(f'{kpla}', param=param)
    model.show_param()    

In [ ]:
keplerian_parameters = {}
for i, key in enumerate(model.keplerian['0']._param):
    keplerian_parameters[key] = model.keplerian['0']._par[i]

linear_parameters = {}
for i, key in enumerate(model._lin_name):
    linear_parameters[key] = model._lin_par[i]
    
print(linear_parameters)    
print(keplerian_parameters)    

## Retrieve coordinates from Gaia DR3

In [ ]:
# get source coordinates from DR3
source_id = simulated_astrometry.loc[0, 'source_id']

query = f"SELECT ra,dec,parallax,pmra,pmdec,ref_epoch FROM gaiadr3.gaia_source WHERE source_id = {source_id}"
job = Gaia.launch_job_async(query=query, verbose=True)
dr3_gaia_source = job.get_results().to_pandas()
display(dr3_gaia_source)

## Retrieve a primary-mass estimate from Gaia DR3

In [ ]:
query_m1 = f"SELECT m1, m2 FROM gaiadr3.binary_masses WHERE source_id = {source_id}"
job_m1 = Gaia.launch_job_async(query=query_m1, verbose=True)
dr3_binary_masses = job_m1.get_results().to_pandas()
display(dr3_binary_masses)

m1_MS_gaia = dr3_binary_masses._get_value(0, 'm1')
print(m1_MS_gaia)

## Create orbit model Plot the fitted orbit and the astrometry

This uses several helper functions and most of the code below is only formatting

In [ ]:
reference_time = Time(simulated_astrometry.loc[0, 'ref_epoch'], format='jyear')
model_name = 'multiple_star'

attribute_dict = OrderedDict([  
                                ('ra', dr3_gaia_source.loc[0,'ra']), 
                                ('dec', dr3_gaia_source.loc[0,'dec']),
    
                                ('absolute_plx_mas', linear_parameters['parallax']), 
                                ('muRA_mas', linear_parameters['mura']),
                                ('muDE_mas', linear_parameters['mudec']),
                                ('muAlphaStar_masPyr', linear_parameters['mura']),
                                ('muDelta_masPyr', linear_parameters['mudec']),
                                ('Tref_MJD', reference_time.mjd),
                                ('scan_angle_definition', 'gaia'),
                             ])  

# compute companion mass corresponding to orbital parameters
#m1_MS = 1.  

# USE QUERIED GAIA m1_MS value <=======================
m1_MS = m1_MS_gaia

a_m = convert_from_angular_to_linear(keplerian_parameters['as'], linear_parameters['parallax'])
m2_kg = pjGet_m2(m1_MS * MS_kg, a_m, keplerian_parameters['P'])
m2_MJ = m2_kg / MJ_kg
    
attribute_dict['m2_MJ'] = m2_MJ
attribute_dict['m1_MS'] = m1_MS
attribute_dict['P_day'] = keplerian_parameters['P']
attribute_dict['p1_omega_deg'] = np.rad2deg(keplerian_parameters['w'])
attribute_dict['ecc'] = keplerian_parameters['e']
attribute_dict['p1_OMEGA_deg'] = np.rad2deg(keplerian_parameters['bigw'])
attribute_dict['p1_incl_deg'] = np.rad2deg(keplerian_parameters['i'])
attribute_dict['p1_Tp_day'] = attribute_dict['Tref_MJD'] + keplerian_parameters['Tp']  # this is in MJD
attribute_dict['p1_a1_mas'] = keplerian_parameters['as']
attribute_dict['p1_ecc'] = keplerian_parameters['e']
attribute_dict['p1_period_day'] = keplerian_parameters['P']
attribute_dict['p1_Tp_day-T0'] = keplerian_parameters['Tp']
attribute_dict['offset_alphastar_mas'] = linear_parameters['ra']
attribute_dict['offset_delta_mas'] = linear_parameters['dec']

attribute_dict['source_id'] = source_id

plot_astrometry(simulated_astrometry, pd.DataFrame(attribute_dict, index=[0]), plot_dir, reference_time, model_name)    